## Calculate (RESP) partial charges with Psi4
import the following modules

In [1]:
import os
import psi4
import resp
from openbabel import openbabel as ob
from rdkit import Chem
from rdkit.Chem import AllChem

### some helper functions

In [2]:
def neutralize_atoms(mol):
    pattern = Chem.MolFromSmarts("[+1!h0!$([*]~[-1,-2,-3,-4]),-1!$([*]~[+1,+2,+3,+4])]")
    at_matches = mol.GetSubstructMatches(pattern)
    at_matches_list = [y[0] for y in at_matches]
    if len(at_matches_list) > 0:
        for at_idx in at_matches_list:
            atom = mol.GetAtomWithIdx(at_idx)
            chg = atom.GetFormalCharge()
            hcount = atom.GetTotalNumHs()
            atom.SetFormalCharge(0)
            atom.SetNumExplicitHs(hcount - chg)
            atom.UpdatePropertyCache()
    return mol

def cleanUp(psi4out_xyz):
    deleteTheseFiles = ['1_default_grid.dat','1_default_grid_esp.dat','grid.dat','timer.dat']
    deleteTheseFiles.append(psi4out_xyz)
    for fileName in deleteTheseFiles:
        if os.path.exists(fileName):
            os.remove(fileName)

def get_xyz_coords(mol):
    if not mol is None:
        num_atoms = mol.GetNumAtoms()
        xyz_string=""
        for counter in range(num_atoms):
            pos=mol.GetConformer().GetAtomPosition(counter)
            xyz_string = xyz_string + ("%s %12.6f %12.6f %12.6f\n" % (mol.GetAtomWithIdx(counter).GetSymbol(), pos.x, pos.y, pos.z) )
    return xyz_string


def calcRESPCharges(mol, basisSet, method, gridPsi4 = 1):
    options = {'BASIS_ESP': basisSet,
               'METHOD_ESP': method,
               'RESP_A': 0.0005,
               'RESP_B': 0.1,
               'VDW_SCALE_FACTORS':[1.4, 1.6, 1.8, 2.0],
               'VDW_POINT_DENSITY':int(gridPsi4)
    }

    resp_charges = resp.resp([mol], options)
    return resp_charges


### Set some variables and stuff

In [3]:
method = 'b3lyp'
basisSet = '3-21g' #'6-31G**'
neutralize = True
psi4.set_memory('10 GB')
obConversion = ob.OBConversion()
obConversion.SetInAndOutFormats("xyz", "mol2")
singlePoint = True
path = "./data"


  Memory set to   9.313 GiB by Python driver.


### Read sdf file (3D) into a list 

In [4]:
inputFile = "./data/txa.sdf"
molList = Chem.SDMolSupplier(inputFile, removeHs=False)

### Loop over compounds in list and calculate partial charges

In [30]:
for mol in molList:
    
    if not mol is None:

        molId = mol.GetProp("_Name")
        print('Trying:', molId)
        
        if neutralize:
            mol = neutralize_atoms(mol)
            mol = Chem.AddHs(mol)

        xyz_string = get_xyz_coords(mol) 
        psi_mol = psi4.geometry(xyz_string)

        ### single point calculation
        outfile_mol2 = inputFile[:-4]+".mol2"

        if singlePoint:
            print('Running singlepoint...')
            resp_charges = calcRESPCharges(psi_mol, basisSet, method, gridPsi4 = 1)

        else:
            print('Running geometry optimization...')            
            methodNbasisSet = method+"/"+basisSet
            psi4.optimize(methodNbasisSet, molecule=psi_mol)
            resp_charges = calcRESPCharges(psi_mol, basisSet, method, gridPsi4 = 1)            

        ### save coords to xyz file 
        psi4out_xyz = molId + '.xyz'
        psi_mol.save_xyz_file(psi4out_xyz,1)

        ### read xyz file and write as mol2 
        ob_mol = ob.OBMol()
        obConversion.ReadFile(ob_mol, psi4out_xyz)

        ### set new partial charges
        count = 0
        for atom in ob.OBMolAtomIter(ob_mol):
            newChg = resp_charges[1][count]
            atom.SetPartialCharge(newChg)
            count += 1

        ### write as mol2 
        outfile_mol2 = path+"/"+molId+"_partialChgs.mol2"
        print("Finished. Saved compound with partial charges as mol2 file: %s" % outfile_mol2)
        obConversion.WriteFile(ob_mol, outfile_mol2)

        ### clean up
        cleanUp(psi4out_xyz)


Trying: TXA
Running singlepoint...

*** tstart() called on pbarletta-xps
*** at Fri Jun 17 09:50:10 2022

   => Loading Basis Set <=

    Name: 3-21G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1          entry N          line    79 file /home/pbarletta/anaconda3/envs/chemi/share/psi4/basis/3-21g.gbs 
    atoms 2, 4, 6-11 entry C          line    68 file /home/pbarletta/anaconda3/envs/chemi/share/psi4/basis/3-21g.gbs 
    atoms 3, 5       entry O          line    90 file /home/pbarletta/anaconda3/envs/chemi/share/psi4/basis/3-21g.gbs 
    atoms 12-26      entry H          line    21 file /home/pbarletta/anaconda3/envs/chemi/share/psi4/basis/3-21g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RKS Reference
                        1 Threads,   9536 MiB Core
         ---------------

### acá hago lo mismo, pero con 1 sóla molécula y en 2 stages, como debe ser (supuestamente)

In [5]:
mol = next(molList)
molId = mol.GetProp("_Name")
print('Trying:', molId)

if neutralize:
    mol = neutralize_atoms(mol)
    mol = Chem.AddHs(mol)

xyz_string = get_xyz_coords(mol) 
psi_mol = psi4.geometry(xyz_string)

### single point calculation
outfile_mol2 = inputFile[:-4]+".mol2"

if singlePoint:
    print('Running singlepoint...')
    resp_charges = calcRESPCharges(psi_mol, basisSet, method, gridPsi4 = 1)

else:
    print('Running geometry optimization...')            
    methodNbasisSet = method+"/"+basisSet
    psi4.optimize(methodNbasisSet, molecule=psi_mol)
    resp_charges = calcRESPCharges(psi_mol, basisSet, method, gridPsi4 = 1)            

Trying: TXA
Running singlepoint...

*** tstart() called on pbarletta-xps
*** at Fri Jun 17 11:18:10 2022

   => Loading Basis Set <=

    Name: 3-21G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1          entry N          line    79 file /home/pbarletta/anaconda3/envs/chemi/share/psi4/basis/3-21g.gbs 
    atoms 2, 4, 6-11 entry C          line    68 file /home/pbarletta/anaconda3/envs/chemi/share/psi4/basis/3-21g.gbs 
    atoms 3, 5       entry O          line    90 file /home/pbarletta/anaconda3/envs/chemi/share/psi4/basis/3-21g.gbs 
    atoms 12-26      entry H          line    21 file /home/pbarletta/anaconda3/envs/chemi/share/psi4/basis/3-21g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RKS Reference
                        1 Threads,   9536 MiB Core
         ---------------

In [6]:
resp_charges[1]

array([-9.62851554e-01, -1.58568061e-03, -6.28763594e-01, -8.22742078e-03,
       -5.79626215e-01, -4.36352729e-02, -7.54726919e-03, -5.61995673e-04,
       -1.15541181e-01,  8.22519833e-02,  6.84958554e-01,  3.74749955e-02,
       -1.73823171e-02, -1.06635531e-02,  3.11417679e-02,  5.46788466e-02,
        2.16484769e-02,  2.47363249e-02,  1.92161441e-02,  6.07121101e-02,
        2.35405911e-02,  8.37639436e-02,  3.78162293e-01,  3.60945445e-01,
        2.91756998e-02,  4.83978878e-01])

#### 2nd stage

##### no intentar correr esto. No lo completé

In [ ]:
options = {'BASIS_ESP': basisSet,
            'METHOD_ESP': method,
            'RESP_A': 0.001,
            'RESP_B': 0.1,
            'VDW_SCALE_FACTORS':[1.4, 1.6, 1.8, 2.0],
            'VDW_POINT_DENSITY':1.0
}

acá hay q decidir q átomos van a estar constrained. 

La 1er constraint global es la carga neta de la molécula (será q `neutralize_atoms()` logra esto neutralizando c/ átomo?).

Luego, c/ átomo equivalente debe tener la constraint de tener la misma carga. Habría q ver como se automatiza esto (detectar automáticamente átomos equivalentes). resp entiende éstas constraints con la opción:
```
options["constraint_group"] = [[7, 9], [8, 10], [1, 4], [2, 3, 5, 6]]
```
[Ejemplo](https://github.com/cdsgroup/resp/blob/master/examples/example4.py)
`

In [9]:
# Éste es otro tipo de constraint. Supongo q fija la carga de
# los átomos 5-8 (4-7), p/ q la 2nd stage sólo fitee al resto.

constraint_charge = []
for i in range(4, 8):
    constraint_charge.append([resp_charges[1][i], [i+1]])
options['constraint_charge'] = constraint_charge

In [10]:
constraint_charge

[[-0.5796262151145127, [5]],
 [-0.04363527292970923, [6]],
 [-0.007547269192765417, [7]],
 [-0.0005619956728749703, [8]]]

In [ ]:
resp_charges2 = resp.resp([mol], options)

In [81]:
### save coords to xyz file 
psi4out_xyz = molId + '.xyz'
psi_mol.save_xyz_file(psi4out_xyz,1)

### read xyz file and write as mol2 
ob_mol = ob.OBMol()
obConversion.ReadFile(ob_mol, psi4out_xyz)

True

##### guarda q openbabel tarda en actualizar las cargas y se terminan guardando las cargas viejas, q no tienen sentido alguno

In [89]:
print([atom.GetPartialCharge() for atom in ob.OBMolAtomIter(ob_mol)])

[-0.9628515540947828, -0.0015856806069533456, -0.628763594351746, -0.008227420780451418, -0.5796262151145127, -0.04363527292970923, -0.007547269192765417, -0.0005619956728749703, -0.11554118096864227, 0.08225198332500387, 0.6849585540125035, 0.0374749954867026, -0.017382317091470242, -0.010663553131782934, 0.031141767879884607, 0.05467884659913111, 0.021648476935749034, 0.024736324916901965, 0.01921614406649013, 0.06071211014924121, 0.023540591107370383, 0.08376394360248336, 0.37816229319452127, 0.36094544500630515, 0.029175699800740546, 0.48397887785266197]


In [85]:
### set new partial charges
for i, atom in enumerate(ob.OBMolAtomIter(ob_mol)):
    atom.SetPartialCharge(resp_charges[1][i])

### write as mol2 
outfile_mol2 = path+"/"+molId+"_partialChgs.mol2"
print("Finished. Saved compound with partial charges as mol2 file: %s" % outfile_mol2)
obConversion.WriteFile(ob_mol, outfile_mol2)

### clean up
cleanUp(psi4out_xyz)

Finished. Saved compound with partial charges as mol2 file: ./data/TXA_partialChgs.mol2


##### actualizó cargas?

In [91]:
print([atom.GetPartialCharge() for atom in ob.OBMolAtomIter(ob_mol)])

[-0.9628515540947828, -0.0015856806069533456, -0.628763594351746, -0.008227420780451418, -0.5796262151145127, -0.04363527292970923, -0.007547269192765417, -0.0005619956728749703, -0.11554118096864227, 0.08225198332500387, 0.6849585540125035, 0.0374749954867026, -0.017382317091470242, -0.010663553131782934, 0.031141767879884607, 0.05467884659913111, 0.021648476935749034, 0.024736324916901965, 0.01921614406649013, 0.06071211014924121, 0.023540591107370383, 0.08376394360248336, 0.37816229319452127, 0.36094544500630515, 0.029175699800740546, 0.48397887785266197]
